In [25]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import re
from sklearn.externals import joblib
from nltk.corpus import stopwords 
nltk.download('stopwords')
import praw
import datetime as dt
import numpy as np
from sklearn.metrics import accuracy_score


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/parasmehan123/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/parasmehan123/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data=pd.read_csv('data_cleaned.csv')

In [3]:
tf=TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7,stop_words=stopwords.words('english'))

In [4]:
tf.fit(data['text'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.7, max_features=1500, min_df=5,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs',... 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"],
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [5]:
stemmer = WordNetLemmatizer()
def clean(X):
    clean_X = []

    for sen in X:  
        # Remove all the special characters
        document = re.sub(r'\W', ' ', str(sen))

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # Converting to Lowercase
        document = document.lower()

        # Lemmatization
        document = document.split()

        document = [stemmer.lemmatize(word) for word in document]
        document = ' '.join(document)

        clean_X.append(document)
    return clean_X

def clean_url(url):   
    url_clean = []
    for sen in url:
        sen=sen.replace('/',' ')
        # Remove all the special characters
        document = re.sub(r'\W', ' ', str(sen))

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # Converting to Lowercase
        document = document.lower()

        # Lemmatization
        document = document.split()

        document = [stemmer.lemmatize(word) for word in document]
        document = ' '.join(document)
        if document=="nan":
            url_clean.append('')
        else:
            url_clean.append(document)
    return url_clean
def clean_master(ti,bo,ur):
    ti_clean=clean(ti)
    bo_clean=clean(bo)
    ur_clean=clean_url(ur)
    text=[]
    for i in range(len(ti)):
        text.append(ti_clean[i]+' '+bo_clean[i]+' '+ur_clean[i])
    return text
    

In [6]:
flair_dict={0:'Political',1:'Non-political',2:'Reddiquette',3:'AskIndia',4:'Science & Technology',5:'Policy & Economy',6:'Finance & Business',7:'Sports and food',8:'Photography',9:'AMA'}

In [7]:
clf = joblib.load('model.joblib') 

In [11]:
def get_flair(u):
    reddit=praw.Reddit(client_id='ST0obmq3HAomHQ',client_secret='ECeazJYbpZ5Kx83g6RrWPDyPO0A',user_agent='Precog Project',username='parasmehan123',password='pMarsMehan@123')
    s=reddit.submission(url=u)
    bo=[s.selftext]
    ti=[s.title]
    ur=[s.url]
    text=clean_master(ti,bo,ur)
    print(text)
    return flair_dict[clf.predict(tf.transform(text))[0]]

In [12]:
get_flair("https://www.reddit.com/r/india/comments/cdryvu/sunil_chhetri/")

['sunil chhetri  http i redd it u3fc46hs8la31 jpg']


'Political'

In [13]:
get_flair("https://www.reddit.com/r/india/comments/cdqtei/lpt_amazonflipkart_sale_its_never_a_good_deal_if/")

['lpt amazon flipkart sale it never a good deal if you were not going to buy that item in the first place if you don t need an item and you are buying it solely for the reason that it is on sale then that wasn t really a good financial decision if you are cautious about your finance if you really need an item then buying it without that item being on sale is still bang for your buck http www reddit com r india comment cdqtei lpt_amazonflipkart_sale_its_never_a_good_deal_if']


'AMA'

In [14]:
get_flair("https://www.reddit.com/r/india/comments/cdkoc8/how_an_indian_tycoon_fought_big_pharma_to_sell/")

['how an indian tycoon fought big pharma to sell aid drug for 1 a day  http qz com india 1666032 how indian pharma giant cipla made aid drug affordable']


'AMA'

In [15]:
get_flair("https://www.reddit.com/r/india/comments/cdrwii/psa_change_region_in_your_xiaomi_phone_to_any/")

['psa change region in your xiaomi phone to any country in europe to get rid of all the bloatware all the shitty apps will disappear they are not uninstalled i believe but the apps don t clutter your menu and there are no notification something is better nothing if you don t want to root your device http www reddit com r india comment cdrwii psa_change_region_in_your_xiaomi_phone_to_any']


'AMA'

In [16]:
get_flair("https://www.reddit.com/r/india/comments/cdifvp/mahua_moitra_files_criminal_defamation_case/")

['mahua moitra file criminal defamation case against sudhir chaudhary  http thewire in law mahua moitra criminal defamation sudhir chaudhary']


'Photography'

In [20]:
fl=pd.read_csv('data.csv')["flair"].tolist()

In [22]:
text=data['text'].tolist()

In [23]:
text=tf.transform(text)

In [26]:
accuracy_score(clf.predict(text),fl)

0.9549202858713579